# Welcome to Exkaldi

In this section, we will train the triphone HMM-GMM up to train_delta step.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Firstly, make a triphone model object.

In [ ]:
model1 = exkaldi.hmm.TriphoneHMM()

model1

___model1___ is unavaliable now. We have to initialize its data. 

We will use thses files which have been generated in early steps.

___tree___ and ___treeStats___ : generated in 6_train_decision_tree

___topo___:  generated in 5_train_mono_HMM-GMM

In [ ]:
treeFile = os.path.join(dataDir, "exp", "train_tree", "tree")
treeStatsFile = os.path.join(dataDir, "exp", "train_tree", "treeStats.acc")
topoFile = os.path.join(dataDir, "exp", "topo")
numgauss = 1000

In [ ]:
model1.initialize(tree=treeFile, treeStatsFile=treeStatsFile, topoFile=topoFile, numgauss=numgauss)

model1.info

The training steps of triphone HMM are almost the same as monophone HMM except that we don't use equally aligning at the first time.

We will introduce the traing step in a nutshell.

### Training in deltail

At the first step, we must generate the new alignment data in the first step. You can convert the lastest alignment data generated by monophone model to a new alignment data corresponding to triphone. Use the generated alignment and final monophone model.

In [ ]:
aliFile = os.path.join(dataDir, "exp", "train_mono", "final.ali")
monoFile= os.path.join(dataDir, "exp", "train_mono", "final.mdl")

In [ ]:
newAli = exkaldi.hmm.convert_alignment(aliFile, monoFile, model1, treeFile)

newAli

In another way, align feature again directly with new triphone model. In the next steps, we will review the steps to train a HMM-GMM model and use it to align acoustic feature.

In [ ]:
del newAli

Prepare a lexicons (generated in 3_prepare_lexicons).

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Prepare int-format transcription (generated in 5_train_mono_HMM-GMM ).

In [ ]:
intTransFile = os.path.join(dataDir, "exp", "text.int")

trans = exkaldi.load_trans(intTransFile)

trans.subset(nHead=1)

Prepare L.fst file (generated in 3_prepare_lexicons).

In [ ]:
Lfile = os.path.join(dataDir, "exp", "L_disambig.fst")

Prepare feature (generated in 2_feature_processing).

In [ ]:
featFile = os.path.join(dataDir, "exp", "mfcc.ark")

feat = exkaldi.load_feat(featFile)

feat

#### 1. Compile new train graph.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta")

exkaldi.utils.make_dependent_dirs(outDir, False)

In [ ]:
trainGraphFile = os.path.join(outDir, "train_graph")

model1.compile_train_graph(tree=treeFile, transcription=trans, LFile=Lfile, outFile=trainGraphFile, lexicons=lexicons)

#### 2. Align acoustic feature.

In [ ]:
ali = model1.align(feat, trainGraphFile, lexicons=lexicons)

ali

#### 3. Accumulate statistics.

In [ ]:
statsFile = os.path.join(outDir, "stats.acc")

model1.accumulate_stats(feat=feat, alignment=ali, outFile=statsFile)

#### 4. Update HMM-GMM parameters.

In [ ]:
targetGaussians = 1100

model1.update(statsFile, targetGaussians)

model1.info

### Training in high-level

In this step, we will introduce how to training the triphone in directly.

In [ ]:
del model1
del ali
del trans

os.remove(trainGraphFile)
os.remove(statsFile)

Some file paths or objects defined above will be used here. 

Firstly, initialize model. Give lexicons as a optional parameter.

In [ ]:
model1 = exkaldi.hmm.TriphoneHMM(lexicons=lexicons)

model1.initialize(tree=treeFile, treeStatsFile=treeStatsFile, topoFile=topoFile, numgauss=numgauss)

model1.info

Then train it.

In [ ]:
outDir = os.path.join(dataDir, "exp", "train_delta")
transFile = os.path.join(dataDir, "train", "text")

model1.train(feat=feat, transcription=transFile, LFile=Lfile, tree=treeFile, tempDir=outDir, 
             num_iters=10, max_iter_inc=8, totgauss=1500)

Final model and alignment have been saved in file automatically. Look the final model information.

In [ ]:
model1.info